In [5]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re

class DenseNet(nn.Module):
    def __init__(self,context_length,embed_size=100):
        super().__init__()
        self.n = context_length*2
        self.embed_size = 100
        self.act = nn.ReLU()
        self.out = nn.Tanh() 
        self.hidden1 = nn.Linear(self.n*self.embed_size,2048)
        self.hidden2 = nn.Linear(2048,512)
        self.hidden3 = nn.Linear(512,self.embed_size)
 
    def forward(self,x):
        x = x.view(x.size(0), -1)
        x = self.act(self.hidden1(x))
        x = self.act(self.hidden2(x))
        x = self.out(self.hidden3(x))
        return x
    


model_to_test =  DenseNet(context_length = 10)
model_to_test.load_state_dict(torch.load('../outputs/2021-08_wiki_only_5words-covid_splendid-sunset-8.pt'))

<All keys matched successfully>

In [6]:
from util import *

#Loading the data
W_norm,vocab,ivocab = load_glove()

In [7]:
random_sent = '<pad> <pad> <pad> <pad> <pad> <pad> emergency use authorization allows covid vaccine distributed us use individuals years age older <pad> '
target_word = 'pneumonia'
target_label = np.array(get_glove_vec(target_word,W_norm,vocab))
random_sent = re.sub('[\n\r\ ]+',' ',random_sent).strip()

sample_tensor = torch.Tensor([[get_glove_vec(word,W_norm,vocab) for word in random_sent.split(' ')]])
sample_output = model_to_test(sample_tensor)
output = sample_output.squeeze(1)
vec_output = output.detach().numpy()
print(f"Test 3: Custom Test\n\n{random_sent}\n\n")
def __distance(W, vocab, ivocab, vec_output):


    dist = np.dot(W, vec_output.T).squeeze(1)
    print(dist.shape)
    a = np.argsort(-dist)[:10]

    print("\n                               Word       Unnormalized Cosine distance\n")
    print("---------------------------------------------------------\n")
    for i,x in enumerate(a):
        print("%d%35s\t\t%f" % (i,ivocab[str(x)], dist[x]))

def cosim(v1,v2):
    return np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

__distance(W_norm,vocab,ivocab,vec_output)
print(f"\n\n\t\tCosim score: {cosim(vec_output,target_label)}")

Test 3: Custom Test

<pad> <pad> <pad> <pad> <pad> <pad> emergency use authorization allows covid vaccine distributed us use individuals years age older <pad>


(400000,)

                               Word       Unnormalized Cosine distance

---------------------------------------------------------

0                            disease		0.972045
1                           diseases		0.861759
2                          infection		0.804020
3                            illness		0.774055
4                              virus		0.764235
5                             cancer		0.760667
6                         infections		0.744311
7                           epidemic		0.741166
8                       tuberculosis		0.734336
9                           infected		0.726196


		Cosim score: [0.7071136]


In [10]:
infer_batch = []
with open('../processed_data/cat_corpus_c10.txt','r',encoding='utf-8') as f:
    sents = f.readlines()
    for line in sents:
        x = re.sub('[\r\n\ ]+',' ',line)
        infer_batch.append(np.array([get_glove_vec(w,W_norm,vocab) for w in x.strip().split(' ')]))
        
sample_infer = torch.tensor(np.array(infer_batch)).float()

sample_output = model_to_test(sample_infer)

a = torch.mean(sample_output,axis = 0).detach().numpy().reshape((1,100))
__distance(W_norm,vocab,ivocab,a)
print(f"\n\n\t\tCosim score: {cosim(a,target_label)}")

(400000,)

                               Word       Unnormalized Cosine distance

---------------------------------------------------------

0                            disease		0.955879
1                           diseases		0.848173
2                          infection		0.794759
3                              virus		0.761556
4                            illness		0.761492
5                             cancer		0.744749
6                           epidemic		0.742815
7                         infections		0.736916
8                       tuberculosis		0.726880
9                                flu		0.724777


		Cosim score: [0.71008273]
